# aiojobs

Since it is difficult to control the running event loop for the ayncio, it is almost impossible to schedule multiple task at the same time without complicated coiding. 

Now this is where aiojobs comes in. without have to go into low-level understanding of coroutine, aiojobs provide few functions easily solve these problems.

*pip install aiojobs*

aiojobs allow for easy concurrency inside coroutine. In this code example, 10 individual task will be running at the same time given to their own time to sleep.

In [ ]:
import asyncio
import aiojobs

async def sleepfor(seconds):
  await asyncio.sleep(seconds)
  print(f"{seconds=}")

async def main():
  scheduler = await aiojobs.create_scheduler()
  for second in range(10):
    task = await scheduler.spawn(sleepfor(second))
  await task.wait()
  print("done")

await main()
# inside jupyter, you dont have to run asyncio.run() to run coroutine
# since jupyter is running in coroutine anyway

For this bot, aiojobs is heavily utilized in SlinkBot.py and Social_Link_Handler.py

scheduler is a class that can manage all tasks created 

for this bot, sceduler is defined in *on_ready()* method inside SlinkBot.py. scheduler can run all scheduled task from lvling_loop concurrently inside Social_Link_Handler.py

# flexibility

aiojobs provides a huge flexibility about creating tasks(or jobs). In the following example, I called scheduler.spawn() within another scheduler.spawn() under the same scheduler. It basically provides simple solution to a huge complicate issue of the voice leveling system within this bot.

In [ ]:
import aiojobs
import asyncio
    
async def counter(scheduler, seconds, index):
    for second in range(seconds):
        task = await scheduler.spawn(counter_sleep(second, index))# spawn inside another spawn
    await task.wait()

async def counter_sleep(seconds, index):
    await asyncio.sleep(seconds)
    print(f"{index}: count: {seconds}")

async def main():
    scheduler = await aiojobs.create_scheduler()

    task = await scheduler.spawn(counter(scheduler, 3, 1))# trigger the loop
    task = await scheduler.spawn(counter(scheduler, 5, 2))

await main()

# Leveling system using aiojobs

this bot is using aiojobs to calculate the leveling for each person inside a voice call.
since the leveling is so complicated, I will try to explain the best I can about the following code.

first, lets see why the leveling system need coroutine:
- when everyone inside a voice call, leveling up is a time problem.
- when everyone inside the voice call has different stats on their level, it will be difficult to track all timer.
- when someone leave the vc, stop the timer for that individual instead of everyone

in order to do those, multitasking will be perfect to manage, thus coroutine is the solution. 

there is also going to be a few scenerio need to consider:
- level up when someone join a vc with 1 or more people inside the vc
- when someone leave vc with 1 or more people inside the vc before level up
- level up when someone inside a vc

keep those problems in mind, now we can start looking at the code structure:
- conn_line object records all necessary information about each two pairs inside a voice call
- whenever a person join a vc, create conn_line object and automatically start the timer between each two pair of person inside a vocie call(can be done by double for looping, can be seen in join_vc_handler() in SlinkBot.py)
- the timer convert the xp each persona have into the time require to level up and run it through asyncio.sleep()
- once the timer reached, triggers the level up, and restart the cycle
- if the user leave before the timer is up, return the time.time() and subtract with the time when conn_line object created

where aiojobs come in is when the timer and the sleep system comes in. I need to develop a sytem where:
- asyncio.sleep() inside a while loop where the loop need to be awaiting task to finish. 
- the while loop can be turn off anytime
- while loop and any other coroutine need to be running concurrently

Finally, we can look at the code below:
- two functions label under counter are solely for debug purposes. it helps you see how long the time has passed since the code run
- global variable switch can break the while loop anytime
- loop body contains the while loop and it will keep spawning loop_coro every 5 second. until the switch turned off
- task.wait() inside the while loop means the while loop will not proceed until the task is finish
- since it is possible to spawn task within spawn(), you can spawn two jobs running the while loop concurrently.

In [5]:

import asyncio
import aiojobs

switch = True

#-- counter for debug purpose
async def counter1(scheduler, seconds):
    for second in range(seconds):
        task = await scheduler.spawn(counter1_sleep(second))
    await task.wait()

async def counter1_sleep(seconds):
    await asyncio.sleep(seconds)
    print(f"coutner: {seconds}")

#-- loop
async def loop_coro(seconds, mult):
    await asyncio.sleep(mult)
    print(f"{mult}: loop: {seconds*mult}")

async def loop(scheduler, mult):
    num = 0
    global switch
    while switch == True:
        task = await scheduler.spawn(loop_coro(num, mult))
        await task.wait()
        num+=1
        if num >= 5:
            break
        if switch == False:
            print("loop break")
            break

async def switchOff():
    global switch
    await asyncio.sleep(7)
    switch = False
    print("switch off")

async def main():
    scheduler = await aiojobs.create_scheduler()

    task = await scheduler.spawn(counter1(scheduler, 20))
    lp_switch = await scheduler.spawn(switchOff())
    lp = await scheduler.spawn(loop(scheduler, 5))
    lp2 = await scheduler.spawn(loop(scheduler, 2))

await main()

coutner: 0
coutner: 1
coutner: 2
2: loop: 0
coutner: 3
coutner: 4
2: loop: 2
coutner: 5
5: loop: 0
coutner: 6
2: loop: 4
switch off
coutner: 7
coutner: 8
2: loop: 6
loop break
coutner: 9
coutner: 10
5: loop: 5
loop break
coutner: 11
coutner: 12
coutner: 13
coutner: 14
coutner: 15
coutner: 16
coutner: 17
coutner: 18
coutner: 19
